In [2]:
pip install fuzzywuzzy

In [3]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
resolved_queries_df = pd.read_csv('resolved_queries.csv')
new_queries_df = pd.read_csv('new_queries.csv')

In [6]:
def preprocess_text(text):
    return ' '.join(text.lower().strip().split())

In [7]:
resolved_queries_df['Pre_Resolved_Query'] = resolved_queries_df['Pre_Resolved_Query'].apply(preprocess_text)
new_queries_df['Variation_Query'] = new_queries_df['Variation_Query'].apply(preprocess_text)


In [8]:
resolved_queries_dict = dict(zip(resolved_queries_df['Query_ID'], resolved_queries_df['Pre_Resolved_Query']))

In [9]:
resolved_queries_list = [(str(k), v) for k, v in resolved_queries_dict.items()]

In [10]:
resolved_query_texts = [v for _, v in resolved_queries_list]
resolved_query_ids = [k for k, _ in resolved_queries_list]

In [12]:
def find_best_fuzzy_match(query, resolved_query_texts, resolved_query_ids, method='ratio'):
    if method == 'ratio':
        match_text, match_score = process.extractOne(query, resolved_query_texts, scorer=fuzz.ratio)
    elif method == 'partial_ratio':
        match_text, match_score = process.extractOne(query, resolved_query_texts, scorer=fuzz.partial_ratio)
    elif method == 'token_sort_ratio':
        match_text, match_score = process.extractOne(query, resolved_query_texts, scorer=fuzz.token_sort_ratio)
    elif method == 'token_set_ratio':
        match_text, match_score = process.extractOne(query, resolved_query_texts, scorer=fuzz.token_set_ratio)
    else:
        raise ValueError("Invalid method specified.")

    match_id = resolved_query_ids[resolved_query_texts.index(match_text)]
    return match_id, match_score

In [13]:
methods = ['ratio', 'partial_ratio', 'token_sort_ratio', 'token_set_ratio']
results = []

In [14]:

for method in methods:
    match_ids = []
    match_scores = []
    for query in new_queries_df['Variation_Query']:
        match_id, match_score = find_best_fuzzy_match(query, resolved_query_texts, resolved_query_ids, method)
        match_ids.append(match_id)
        match_scores.append(match_score)

    new_queries_df[f'Match_ID_{method}'] = match_ids
    new_queries_df[f'Match_Score_{method}'] = match_scores

    # Store results for evaluation
    results.append((method, new_queries_df[f'Match_Score_{method}'].mean()))

In [15]:
print("Fuzzy Matching Results:")
for method, score in results:
    print(f"{method}: {score}")

Fuzzy Matching Results:
ratio: 64.35
partial_ratio: 65.55
token_sort_ratio: 70.55
token_set_ratio: 75.5


In [16]:
vectorizer_bow = CountVectorizer().fit(resolved_query_texts)
resolved_bow_matrix = vectorizer_bow.transform(resolved_query_texts)

# Vectorize the queries using TF-IDF
vectorizer_tfidf = TfidfVectorizer().fit(resolved_query_texts)
resolved_tfidf_matrix = vectorizer_tfidf.transform(resolved_query_texts)

In [17]:
def find_best_cosine_match(query, vectorizer, resolved_matrix):
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, resolved_matrix).flatten()
    best_match_idx = cosine_similarities.argmax()
    best_match_score = cosine_similarities[best_match_idx]
    return resolved_query_ids[best_match_idx], best_match_score

In [18]:
bow_match_ids = []
bow_match_scores = []
tfidf_match_ids = []
tfidf_match_scores = []

In [19]:
for query in new_queries_df['Variation_Query']:
    # Bag of Words
    bow_match_id, bow_match_score = find_best_cosine_match(query, vectorizer_bow, resolved_bow_matrix)
    bow_match_ids.append(bow_match_id)
    bow_match_scores.append(bow_match_score)

    # TF-IDF
    tfidf_match_id, tfidf_match_score = find_best_cosine_match(query, vectorizer_tfidf, resolved_tfidf_matrix)
    tfidf_match_ids.append(tfidf_match_id)
    tfidf_match_scores.append(tfidf_match_score)

In [20]:
new_queries_df['Match_ID_BoW'] = bow_match_ids
new_queries_df['Match_Score_BoW'] = bow_match_scores
new_queries_df['Match_ID_TFIDF'] = tfidf_match_ids
new_queries_df['Match_Score_TFIDF'] = tfidf_match_scores

# Calculate the average match scores for BoW and TF-IDF
bow_avg_score = np.mean(bow_match_scores)
tfidf_avg_score = np.mean(tfidf_match_scores)

In [21]:
print("\nBoW and TF-IDF Results:")
print(f"BoW Average Score: {bow_avg_score}")
print(f"TF-IDF Average Score: {tfidf_avg_score}")


BoW and TF-IDF Results:
BoW Average Score: 0.6567198566742267
TF-IDF Average Score: 0.6708402353184898


In [22]:
#part 2
import re

In [23]:

base_names_df = pd.read_csv('base_names.csv')
name_variations_df = pd.read_csv('name_variations.csv')

In [24]:
def preprocess_name(name):
    # Remove punctuation
    name = re.sub(r'[^\w\s]', '', name)
    # Convert to lowercase and remove extra spaces
    return ' '.join(name.lower().strip().split())

In [25]:
base_names_df['Base_Name'] = base_names_df['Base_Name'].apply(preprocess_name)
name_variations_df['Variation'] = name_variations_df['Variation'].apply(preprocess_name)


In [26]:
# Create a dictionary of base names for fuzzy matching
base_names_dict = dict(zip(base_names_df['Base_Name_ID'], base_names_df['Base_Name']))


In [27]:
# Convert the dictionary to a list of base name texts and their IDs
base_name_texts = [v for _, v in base_names_dict.items()]
base_name_ids = [k for k, _ in base_names_dict.items()]

In [28]:
# Function to find the best match using fuzzy matching
def find_best_fuzzy_name_match(name_variation, base_name_texts, base_name_ids, method='token_set_ratio'):
    if method == 'ratio':
        match_text, match_score = process.extractOne(name_variation, base_name_texts, scorer=fuzz.ratio)
    elif method == 'partial_ratio':
        match_text, match_score = process.extractOne(name_variation, base_name_texts, scorer=fuzz.partial_ratio)
    elif method == 'token_sort_ratio':
        match_text, match_score = process.extractOne(name_variation, base_name_texts, scorer=fuzz.token_sort_ratio)
    elif method == 'token_set_ratio':
        match_text, match_score = process.extractOne(name_variation, base_name_texts, scorer=fuzz.token_set_ratio)
    else:
        raise ValueError("Invalid method specified.")

    # Find the index of the matched text to get the corresponding ID
    match_id = base_name_ids[base_name_texts.index(match_text)]
    return match_id, match_score

In [29]:
# Apply fuzzy matching using the best method (e.g., 'token_set_ratio') and store the results
name_match_ids = []
name_match_scores = []

for name_variation in name_variations_df['Variation']:
    match_id, match_score = find_best_fuzzy_name_match(name_variation, base_name_texts, base_name_ids, method='token_set_ratio')
    name_match_ids.append(match_id)
    name_match_scores.append(match_score)

In [30]:
name_variations_df['Matched_Base_Name_ID'] = name_match_ids
name_variations_df['Match_Score'] = name_match_scores


print(name_variations_df.head())

      Variation Matches_With_Base_Name  Matched_Base_Name_ID  Match_Score
0   thomas king            Thomas King                    15          100
1    thomasking            Thomas King                    15           57
2  maria garcia           Maria Garcia                     4          100
3     marylewis             Mary Lewis                    12           53
4       nancy w           Nancy Wright                    16           83
